In [1]:
# importing libraries
from os import getcwd, path
from pathlib import Path

from pandas import Series, DataFrame
import pandas as pd
import numpy as np
from typing import Set, List

from data_analysis_helper_fun import *


## Loading data

In [2]:
 # relative path to data directory
data_dir = path.join(Path(getcwd()).parent.absolute(), "data")
data_set_name = 're_data_20231105_process_results'
data_file_name = f'{data_set_name}.csv.tar.gz'
re_data = load_re_data(data_dir, data_file_name)

In [3]:
print('\033[1m' + 'General information: ' + '\033[0m')
print("   Amount of rows:" , len(re_data))
print('   Number of dialectical structures:', len(re_data['ds'].unique()))
print('   Overall number of initial commitments:', len(re_data['init_coms'].unique()))
print('   Sentence-pools:' , re_data['n_sentence_pool'].unique())
print('   Models:' , re_data['model_name'].unique())
print('\n')

General information: 
   Amount of rows: 197
   Number of dialectical structures: 4
   Overall number of initial commitments: 2
   Sentence-pools: [6 7]
   Models: ['GlobalNumpyReflectiveEquilibrium'
 'StandardGlobalReflectiveEquilibriumLinearG'
 'LocalStandardReflectiveEquilibrium'
 'StandardLocalReflectiveEquilibriumLinearG']




## Calculating cases

In [4]:
# the original helper functions
# labelling cases    
def case_names(x, go_or_fp):
    
    # global optima or fixed points?
    if go_or_fp == 'fp':
        column = 'fp_coms_consistent'
        n = x['n_fixed_points']
        
    elif go_or_fp == 'go':
        column = 'go_coms_consistent'
        n = x['n_global_optima']
    
    cases = []
    
    for i in range(n):
    
        if x['init_coms_dia_consistent']  and (not x[column][i]):
            cases.append('very_bad')

        elif x['init_coms_dia_consistent']  and x[column][i]:
            cases.append('good')

        elif (not x['init_coms_dia_consistent'])  and (not x[column][i]):
            cases.append('bad')

        elif (not x['init_coms_dia_consistent'])  and (x[column][i]):
            cases.append('very_good')
            
    return cases

def count_case(x, go_or_fp, case_name):
        # global optima or fixed points?
    if go_or_fp == 'fp':
        column = 'fp_cases'
        
    elif go_or_fp == 'go':
        column = 'go_cases'

    return x[column].count(case_name)

re_data['go_cases'] = re_data.apply(lambda row: case_names(row, 'go'), axis = 1)
re_data['fp_cases'] = re_data.apply(lambda row: case_names(row, 'fp'), axis = 1)

# alternative suggestion:
# we already use col names for counting since we later "group-by.sum()" anyway
re_data['fp_n_very_good_cases'] = re_data.apply(lambda row: (not row['init_coms_dia_consistent']) and row['fixed_point_coms_consistent'], axis = 1)
re_data['fp_n_good_cases'] = re_data.apply(lambda row: row['init_coms_dia_consistent'] and row['fixed_point_coms_consistent'], axis = 1)
re_data['fp_n_bad_cases'] = re_data.apply(lambda row: (not row['init_coms_dia_consistent']) and (not row['fixed_point_coms_consistent']), axis = 1)
re_data['fp_n_very_bad_cases'] = re_data.apply(lambda row: row['init_coms_dia_consistent'] and (not row['fixed_point_coms_consistent']), axis = 1)


display(re_data.head(6)[['fixed_point_coms', 'global_optima', 'go_coms_consistent', 'fp_coms_consistent','go_cases', 'fp_cases', 
                         'fp_n_very_good_cases', 'fp_n_good_cases', 'fp_n_bad_cases', 'fp_n_very_bad_cases' ]])

,fixed_point_coms,global_optima,go_coms_consistent,fp_coms_consistent,go_cases,fp_cases,fp_n_very_good_cases,fp_n_good_cases,fp_n_bad_cases,fp_n_very_bad_cases
0,"{1, 2, 3, 5, -6, -4}","[({1, 5}, {1, 2, 3, 5, -6, -4})]",[True],[True],[good],[good],False,True,False,False
1,"{1, 3, 5, -6, -4}","[({1, 5}, {1, 2, 3, 5, -6, -4}), ({1, 5}, {1, ...","[True, True]","[True, True]","[good, good]","[good, good]",False,True,False,False
2,"{1, 2, 3, 5, -6, -4}","[({1, 5}, {1, 2, 3, 5, -6, -4}), ({1, 5}, {1, ...","[True, True]","[True, True]","[good, good]","[good, good]",False,True,False,False
3,"{1, 3, 5, -6, -4}","[({1, 5}, {1, 3, 5, -6, -4})]",[True],[True],[good],[good],False,True,False,False
4,"{2, 3, 5, 6, -4, -1}","[({2, -4}, {1, 2, 3, 6, -5, -4}), ({2, -4}, {2...","[False, False]","[False, False]","[bad, bad]","[bad, bad]",False,False,True,False
5,"{1, 2, 3, 6, -5, -4}","[({2, -4}, {1, 2, 3, 6, -5, -4}), ({2, -4}, {2...","[False, False]","[False, False]","[bad, bad]","[bad, bad]",False,False,True,False


In [6]:
result_df = re_data[['model_name', 'fp_n_very_good_cases', 'fp_n_good_cases', 'fp_n_bad_cases', 'fp_n_very_bad_cases']].groupby(['model_name']).agg('sum')
result_df['n_fixed_points'] = result_df.apply(lambda x: sum(x) , axis=1)
display(result_df)


,fp_n_very_good_cases,fp_n_good_cases,fp_n_bad_cases,fp_n_very_bad_cases,n_fixed_points
model_name,,,,,
GlobalNumpyReflectiveEquilibrium,10,4,31,0,45
LocalStandardReflectiveEquilibrium,3,8,35,2,48
StandardGlobalReflectiveEquilibriumLinearG,11,46,13,4,74
StandardLocalReflectiveEquilibriumLinearG,18,6,6,0,30
